In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from osgeo import gdal
import glob
import netCDF4
from datetime import datetime

## Mosaic tiles

In [ ]:
path_folder_tiles = ''
output_folder_path = ''

In [ ]:
tile_path = path_folder_tiles+'\\MCD12Q1.A{0}001.{1}.*.*.hdf'

tiles_interest = ['h{}v{}'.format(str(h).zfill(2),str(v).zfill(2)) for h in range(36) for v in range(18)]

wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

tile_size = 2400

lat_max = 80.
lat_min = -80.
lon_max = 180.
lon_min = -180.

res = 0.005

x_size = int((lon_max - lon_min)/res)
y_size = int((lat_max - lat_min)/res)

geot = [lon_min - res/2, res, 0., lat_max + res/2, 0., -1*res] 

In [ ]:

for year in range(2001,2023):

    dst = gdal.GetDriverByName('MEM').Create('', x_size, y_size, 1, gdal.GDT_Int16)
    dst.GetRasterBand(1).WriteArray(np.ones((y_size, x_size), dtype=np.int16) * np.nan)
    dst.SetGeoTransform(geot)
    dst.SetProjection(wgs84_wkt)

    for tile in tiles_interest:
        print(year, tile)

        fname_list = glob.glob(tile_path.format(year,tile))
        
        if len(fname_list) > 0 :
            fname = fname_list[0]

            tile = xr.open_dataset(fname,engine='netcdf4')

            tile_gdal = gdal.Open('HDF4_EOS:EOS_GRID:"{}":MCD12Q1:LC_Type1'.format(fname))  

            src = gdal.GetDriverByName('MEM').Create('', tile_size, tile_size, 1, gdal.GDT_Int16)
            src.SetGeoTransform(tile_gdal.GetGeoTransform())
            src.SetProjection(tile_gdal.GetProjection())
            src.GetRasterBand(1).WriteArray(tile.LC_Type1.data)

            gdal.ReprojectImage(src, dst, None, None, gdal.GRA_NearestNeighbour)

            del src

    with netCDF4.Dataset('{}\\world_LC-IGBP_modis_mosaic_{}.nc'.format(output_folder_path, year), 'w', format='NETCDF4_CLASSIC') as ds:

        t_dim = ds.createDimension('time', 1)
        x_dim = ds.createDimension('longitude', x_size)
        y_dim = ds.createDimension('latitude', y_size)

        time_var = ds.createVariable('time', 'f8', ('time',))
        time_var[:] = netCDF4.date2num([datetime(year,1,1)], units='seconds since 1970-01-01 00:00:00.0', calendar='standard')

        lon_var = ds.createVariable('longitude', 'f8', ('longitude',))
        lon_var[:] = np.linspace(lon_min, lon_max-res, num=x_size)
        
        lat_var = ds.createVariable('latitude', 'f8', ('latitude',))
        lat_var[:] = np.linspace(lat_max, lat_min+res, num=y_size)
        
        lc_var = ds.createVariable('MCD12Q1:LC_Type1', 'i4', ('time', 'latitude', 'longitude'), fill_value=-9999)
        lc_var[:] = dst.GetRasterBand(1).ReadAsArray()

    del dst



## Extract LC data

In [ ]:
path_to_db_folder = '' #string path to folder where database is
ds = pd.read_excel('{0}\\Globe-LFMC-2.0.xlsx'.format(path_to_db_folder), sheet_name='LFMC data')

In [ ]:
globelfmc = ds.copy()
globelfmc

In [ ]:
def year_lc(year): 

    if year < 2001:
        year = 2001

    elif year > 2022:
        year = 2022

    return year

In [ ]:
globelfmc['IGBP Land Cover'] = np.nan
globelfmc['IGBP Land Cover ID'] = np.nan
globelfmc['Year of Land Cover'] = globelfmc['Sampling date (YYYYMMDD)'].map(lambda x: year_lc(x.year))

In [ ]:
lc_names = {1:'Evergreen Needleleaf Forests',
            2:'Evergreen Broadleaf Forests',
            3:'Deciduous Needleleaf Forests',
            4:'Deciduous Broadleaf Forests',
            5:'Mixed Forests',
            6:'Closed Shrublands',
            7:'Open Shrublands',
            8:'Woody Savannas',
            9:'Savannas',
            10:'Grasslands',
            11:'Permanent Wetlands',
            12:'Croplands',
            13:'Urban and Built-up Lands',
            14:'Cropland/Natural Vegetation Mosaics',
            15:'Permanent Snow and Ice',
            16:'Barren',
            17:'Water Bodies'
            }


In [ ]:
path_to_lc_mosaics = '' #mosaics are NetCDF files in lat/lon (WGS84), with 0.005 degrees resolution, dimensions: longitude from -180 to 179.995, latitude from -80 to 79.995

for year in range(2001,2023):
    print(year)

    lc = xr.open_dataset('{0}\\world_LC-IGBP_modis_mosaic_{1}.nc'.format(path_to_lc_mosaics,year))

    sub_sites = sorted(set(globelfmc.loc[globelfmc['Year of Land Cover']==year,'Site name']))

    for site in sub_sites:

        lat = globelfmc.loc[globelfmc['Site name']==site,'Latitude (WGS84, EPSG:4326)'].values[0]
        lon = globelfmc.loc[globelfmc['Site name']==site,'Longitude (WGS84, EPSG:4326)'].values[0]

        lc_value = lc.sel(latitude=lat, longitude=lon, method='nearest')['MCD12Q1:LC_Type1'].data[0]

        globelfmc.loc[(globelfmc['Year of Land Cover']==year) & (globelfmc['Site name']==site),'IGBP Land Cover ID'] = lc_value

        if lc_value in lc_names:
            globelfmc.loc[(globelfmc['Year of Land Cover']==year) & (globelfmc['Site name']==site),'IGBP Land Cover'] = lc_names[lc_value]
        else:
            globelfmc.loc[(globelfmc['Year of Land Cover']==year) & (globelfmc['Site name']==site),'IGBP Land Cover'] = 'Unclassified'

    globelfmc.to_csv('{0}\\Globe-LFMC-2.0_LC_up to {1}.csv'.format(path_to_db_folder, year), index=False)